<a href="https://colab.research.google.com/github/JaiswalFelipe/ITNPAI1_Assignment/blob/main/ImageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INITIAL

In [2]:
import tensorflow as tf
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [80]:
# Train, val, test split
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=1) 

print("Training set size:", X_train.shape)
print("Training set size:", X_val.shape)
print("Testing set size:", X_test.shape)

Training set size: (40000, 32, 32, 3)
Training set size: (10000, 32, 32, 3)
Testing set size: (10000, 32, 32, 3)


In [ ]:
# Define the labels of the dataset
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
          'dog', 'frog', 'horse', 'ship', 'truck']

# Let's view more images in a grid format
# Define the dimensions of the plot grid 

L_grid = 10

# subplot return the figure object and axes object
# we can use the axes object to plot specific figures at various locations

fig, axes = plt.subplots(L_grid, figsize = (15,15))

axes = axes.ravel() # flaten the 15 x 15 matrix into 225 array

n_train = len(X_train) # get the length of the train dataset

# Select a random number from 0 to n_train
for i in np.arange(0, L_grid): # create evenly spaces variables 

    # Select a random number
    index = np.random.randint(0, n_train)
    # read and display an image with the selected index    
    axes[i].imshow(X_train[index,10:])
    class_index = int(y_train[index])
    axes[i].set_title(classes[class_index], fontsize = 10)
    axes[i].axis('off')

In [ ]:
# Normalize data: divide each pixel value by 255 for each channel R,G,B 
# to normalize it to 0-1 range

X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

print(X_train[0].shape)
print(X_train[0])

# Training

In [ ]:
#!pip install d2l==0.16.2 # Uncomment and run this if you are using Google Colab

In [95]:
# AlexNet
import tensorflow as tf
from d2l import tensorflow as d2l

def net():
  return tf.keras.models.Sequential([
    # Here, we use a larger 11 x 11 window to capture objects. At the same
    # time, we use a stride of 4 to greatly reduce the height and width of
    # the output. Here, the number of output channels is much larger than
    # that in LeNet
    tf.keras.layers.Conv2D(filters=96, kernel_size=11, strides=4,

    activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
    # Make the convolution window smaller, set padding to 2 for consistent
    # height and width across the input and output, and increase the
    # number of output channels
    tf.keras.layers.Conv2D(filters=256, kernel_size=5, padding='same',

    activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
    # Use three successive convolutional layers and a smaller convolution
    # window. Except for the final convolutional layer, the number of

    # output channels is further increased. Pooling layers are not used to
    # reduce the height and width of input after the first two
    # convolutional layers
    tf.keras.layers.Conv2D(filters=384, kernel_size=3, padding='same',

    activation='relu'),

    tf.keras.layers.Conv2D(filters=384, kernel_size=3, padding='same',

    activation='relu'),

    tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same',

    activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
    tf.keras.layers.Flatten(),
    # Here, the number of outputs of the fully-connected layer is several
    # times larger than that in LeNet. Use the dropout layer to mitigate
    # overfitting
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    # Output layer. Since we are using Fashion-MNIST, the number of
    # classes is 10, instead of 1000 as in the paper
    tf.keras.layers.Dense(10)])


In [93]:
class TrainCallback(tf.keras.callbacks.Callback): # Save
  """A callback to visiualize the training progress."""
  def __init__(self, net, train_iter, test_iter, num_epochs, device_name):
    self.timer = d2l.Timer()
    self.animator = d2l.Animator(
      xlabel='epoch', xlim=[1, num_epochs],
      legend=['train loss', 'train acc', 'test acc'])
    self.net = net
    self.train_iter = train_iter
    self.test_iter = test_iter
    self.num_epochs = num_epochs
    self.device_name = device_name

def on_epoch_begin(self, epoch, logs=None):
  self.timer.start()

def on_epoch_end(self, epoch, logs):
    self.timer.stop()
    test_acc = self.net.evaluate(self.test_iter, verbose=0,
    return_dict=True)['accuracy']
    
    metrics = (logs['loss'], logs['accuracy'], test_acc)
    self.animator.add(epoch + 1, metrics)
    if epoch == self.num_epochs - 1:
      batch_size = next(iter(self.train_iter))[0].shape[0]
      num_examples = batch_size * tf.data.experimental.cardinality(
      self.train_iter).numpy()
      print(f'loss {metrics[0]:.3f}, train acc {metrics[1]:.3f}, '
        f'test acc {metrics[2]:.3f}')
      print(f'{num_examples / self.timer.avg():.1f} examples/sec on '
        f'{str(self.device_name)}')

# Save
def train_ch6(net_fn, train_iter, test_iter, num_epochs, lr, device):
  device_name = device._device_name
  strategy = tf.distribute.OneDeviceStrategy(device_name)
  with strategy.scope():
    optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    net = net_fn()
    net.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
  callback = TrainCallback(net, train_iter, test_iter, num_epochs,device_name)

  net.fit(train_iter, epochs=num_epochs, verbose=0, callbacks=[callback])
  return net

In [ ]:
batch_size = 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)